In [3]:
import os
import io
from google.cloud import bigquery
from google.oauth2 import service_account

In [18]:
# pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

In [4]:
clean_files = "/Users/natebender/Desktop/repo/footprints/footprints_audience/data"
os.listdir(clean_files)

['20220906_000000_trailrunningtweets.csv',
 '20220216_000000_trailrunningtweets.csv',
 '20220105_0112_000000_trailrunningtweets.csv',
 '20220104_000000_trailrunningtweets.csv',
 '1228-1231_000000_trailrunningtweets.csv',
 '20220131_000000_trailrunningtweets.csv',
 '20220116_0123_000000_trailrunningtweets.csv']

In [5]:
# These first two values will be different on your machine. 
service_path = "C:\\Users\\natebender\\Desktop\\repo\\footprints\\footprints_audience"
service_file = 'footprintsrunning-eb472bf696a0.json' # change this to your authentication information  
gbq_proj_id = 'footprintsrunning' # change this to your poroject. 

# And this should stay the same. 
private_key = service_path + service_file

In [6]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_file)

In [7]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [8]:
# You should have data in the file that you can see via the console. Let's test it here.
query = (
    "SELECT * "
    "FROM `footprintsrunning.twitter_scraping.trail_running`" # change to *your*` project ID, data set ID, and table name! 
)

results = client.query(query)

print("Let's print it ugly.")
print(list(results))

Let's print it ugly.
[]


### Upload data

In [10]:
dataset_ref = client.dataset("twitter_scraping") # set up references to the dataset and table.
table_ref = dataset_ref.table("trail_running") # use your table name

In [16]:
job_config = bigquery.LoadJobConfig() 
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True
input_file = "data/20220906_000000_trailrunningtweets.csv"

In [17]:
with open(input_file, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    
job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_ref, table_ref))

BadRequest: 400 Provided Schema does not match Table footprintsrunning:twitter_scraping.trail_running. Cannot add fields (field: string_field_0)